In [ ]:
!pip install catboost
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.6 MB/s eta 0:00:00


In [83]:
#import packages

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
import matplotlib.pyplot as plt
import seaborn as sea
from lightgbm import LGBMRegressor

In [85]:
# read input files

train = pd.read_csv('train.csv', index_col="id")
test = pd.read_csv('test.csv')

In [86]:
train.head()

,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
id,,,,,,,,,,,,,,,,
0,8.61,3.0,2.0,2.0,2.0,10.30,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0,62.09
1,5.00,2.0,4.0,0.0,3.0,6.66,1.0,0.0,1.0,28206.0,1.0,0.0,0.0,0.0,0.0,121.80
2,14.08,4.0,0.0,0.0,3.0,21.30,1.0,0.0,26.0,21215.0,1.0,0.0,0.0,0.0,0.0,83.51
3,4.02,3.0,5.0,0.0,0.0,14.80,0.0,1.0,36.0,21215.0,1.0,0.0,0.0,0.0,0.0,66.78
4,2.13,3.0,5.0,0.0,3.0,17.00,1.0,1.0,20.0,27694.0,1.0,1.0,1.0,1.0,1.0,111.51


**"prepared_food" is dropped according to the analysis done in the previous attempt**

In [87]:
x_train = train.drop(["cost", "prepared_food"], axis=1)
x_test = test.drop(["id", "prepared_food"], axis=1)

y_train = train[["cost"]]

In [88]:
y_train = np.log1p(y_train)

**Checking Accuracies with Feature Engineering**

In [89]:
columns = x_train.columns
columns = columns.drop("gross_weight")

In [90]:
x_train['store_options'] = x_train[['coffee_bar', 'video_store', 'salad_bar', 'florist']].sum(axis=1)
x_test['store_options'] = x_test[['coffee_bar', 'video_store', 'salad_bar', 'florist']].sum(axis=1)

In [91]:
x_train["children_ratio"] = x_train["total_children"]/x_train["num_children_at_home"]
x_train["children_ratio"] = x_train["children_ratio"].replace([np.inf, -np.inf], 10)
x_train = x_train.fillna(0)

x_test["children_ratio"] = x_test["total_children"]/x_test["num_children_at_home"]
x_test["children_ratio"] = x_test["children_ratio"].replace([np.inf, -np.inf], 10)
x_test = x_test.fillna(0)

In [92]:
x_train["units_per_sqft"] = x_train["unit_sales(in millions)"]/x_train["store_sqft"]

x_test["units_per_sqft"] = x_test["unit_sales(in millions)"]/x_test["store_sqft"]

In [93]:
x_train.head()

,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,florist,store_options,children_ratio,units_per_sqft
id,,,,,,,,,,,,,,,,,
0,8.61,3.0,2.0,2.0,2.0,10.30,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000082
1,5.00,2.0,4.0,0.0,3.0,6.66,1.0,0.0,1.0,28206.0,1.0,0.0,0.0,0.0,1.0,10.0,0.000071
2,14.08,4.0,0.0,0.0,3.0,21.30,1.0,0.0,26.0,21215.0,1.0,0.0,0.0,0.0,1.0,0.0,0.000189
3,4.02,3.0,5.0,0.0,0.0,14.80,0.0,1.0,36.0,21215.0,1.0,0.0,0.0,0.0,1.0,10.0,0.000141
4,2.13,3.0,5.0,0.0,3.0,17.00,1.0,1.0,20.0,27694.0,1.0,1.0,1.0,1.0,4.0,10.0,0.000108


In [94]:
models = {
    "cat_boost" : CatBoostRegressor(verbose=0),
    "lgbm" : LGBMRegressor(),
    "xgb" : XGBRegressor(),
}

In [96]:
scores = []
for model_name, model in models.items():
    fold_scores = cross_val_score(model, x_train, y_train['cost'], cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
    scores.append({
        'model' : model_name,
        'score' : np.mean(fold_scores)
    })

df = pd.DataFrame(scores, columns = ['model', 'score'])
df

,model,score
0,cat_boost,-0.298291
1,lgbm,-0.299314
2,xgb,-0.298412


In [97]:
model_data = {
    'xgb' : {
        'model':XGBRegressor(),
        'params' : {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2, 0.3],
            'max_depth': [3, 4, 5, 6],
            'min_child_weight': [1, 2, 3, 4],
            'gamma': [0, 0.1, 0.2, 0.3]
        }
    },
    'lgbm' : {
        'model':LGBMRegressor(),
        'params' : {
            'categorical_feature' : [1, 2, 3],
            'n_estimators': [450],
            'num_leaves': [50],
            'learning_rate': [0.01, 0.1],
            'max_depth': [5, 10, 15],
            'min_child_samples': [1, 5],
            'min_child_weight' : [10.0],
            'random_state' : [1]
        }
    },
    'cat_boost' : {
        'model': CatBoostRegressor(verbose=0),
        'params' : {
            'iterations': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2, 0.3],
            'depth': [4, 6, 8, 10],
            'l2_leaf_reg': [1, 3, 5, 7]
        }
    }
}

In [ ]:
accuracy_scores = []

for model_name, model_info in model_data.items():
    clf = RandomizedSearchCV(
        estimator = model_info['model'],
        param_distributions = model_info['params'],
        cv = 5,
        n_iter = 50,
        verbose = 2,
        n_jobs = -1,
        scoring="neg_root_mean_squared_error"
    )
    clf.fit(x_train, y_train['cost'])
    accuracy_scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_
    })

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 36 is smaller than n_iter=50. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [ ]:
accuracy_scores

In [ ]:
lgbm_regressor = LGBMRegressor(
    categorical_feature=3,
    learning_rate=0.1,
    min_child_samples=1,
    min_child_weight=10.0,
    n_estimators=450,
    num_leaves=50,
    random_state=1
)

cat_boost_regressor = CatBoostRegressor(
    learning_rate = 0.1,
    l2_leaf_reg = 7,
    iterations = 300,
    depth = 8
)

xgb_regressor = XGBRegressor(
    n_estimators = 300,
    min_child_weight = 3,
    max_depth = 6,
    learning_rate = 0.1,
    gamma = 0.1
)

estimators = [
    ('lgbm', lgbm_regressor),
    ('cat', cat_boost_regressor),
    ('xgb', xgb_regressor)
]


In [ ]:
stacked_model = StackingRegressor(
    estimators=estimators
)

stacked_model.fit(x_train, y_train['cost'])

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1811: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.065312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 695
[LightGBM] [Info] Number of data points in the train set: 360336, number of used features: 17
[LightGBM] [Info] Start training from score 4.563348
0:	learn: 0.3148396	total: 154ms	remaining: 46.1s
1:	learn: 0.3133068	total: 291ms	remaining: 43.4s
2:	learn: 0.3120241	total: 436ms	remaining: 43.2s
3:	learn: 0.3109103	total: 579ms	remaining: 42.8s
4:	learn: 0.3100608	total: 728ms	remaining: 43s
5:	learn: 0.3092677	total: 858ms	remaining: 42s
6:	learn: 0.3086220	total: 943ms	remaining: 39.5s
7:	learn: 0.3080829	total: 1.02s	remaining: 37.1s
8:	learn: 0.3076804	total: 1.08s	remaining: 35s
9:	learn: 0.3072759	total: 1.15s	remaining: 33.5s
10:	learn: 0.3068

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1811: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032987 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 692
[LightGBM] [Info] Number of data points in the train set: 288268, number of used features: 17
[LightGBM] [Info] Start training from score 4.563347


/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1811: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


...

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1811: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 694
[LightGBM] [Info] Number of data points in the train set: 288269, number of used features: 17
[LightGBM] [Info] Start training from score 4.563365


/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1811: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030984 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 690
[LightGBM] [Info] Number of data points in the train set: 288269, number of used features: 17
[LightGBM] [Info] Start training from score 4.563505
0:	learn: 0.3149889	total: 57.8ms	remaining: 17.3s
1:	learn: 0.3134901	total: 112ms	remaining: 16.7s
2:	learn: 0.3122527	total: 164ms	remaining: 16.2s
3:	learn: 0.3111876	total: 217ms	remaining: 16s
4:	learn: 0.3103694	total: 272ms	remaining: 16s
5:	learn: 0.3095843	total: 329ms	remaining: 16.1s
6:	learn: 0.3089374	total: 380ms	remaining: 15.9s
7:	learn: 0.3084230	total: 430ms	remaining: 15.7s
8:	learn: 0.3079644	total: 485ms	remaining: 15.7s
9:	learn: 0.3075400	total: 537ms	remaining: 15.6s
10:	learn: 0.3

StackingRegressor(estimators=[('lgbm',
                               LGBMRegressor(categorical_feature=3,
                                             min_child_samples=1,
                                             min_child_weight=10.0,
                                             n_estimators=450, num_leaves=50,
                                             random_state=1)),
                              ('cat',
                               <catboost.core.CatBoostRegressor object at 0x7b02f3e275e0>),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_byt...
                                            feature_types=None, gamma=0.1,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None, min_child_weight=3,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=300, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))])

In [ ]:
def predict__values(model, x_values):
    y_pred = model.predict(x_values)
    return np.expm1(y_pred)


In [ ]:
y_pred = predict__values(stacked_model, x_test)
final_df = pd.DataFrame( y_pred, columns=['cost'])
final_df.insert(0, 'id', test['id'])
final_df

,id,cost
0,360336,90.868734
1,360337,88.649141
2,360338,89.759162
3,360339,99.959190
4,360340,76.330761
...,...,...
240219,600555,99.483361
240220,600556,90.138789
240221,600557,92.913215
240222,600558,103.204606


In [ ]:
final_df.to_csv('UOM_190215X.csv', index=False)